# '''PyConLatam'''
Queremos agendar las distintas conferencias y talleres que se impartirán en el PyConLatam

In [1]:
from ortools.sat.python import cp_model
import pandas as pd

In [2]:
#Generando instancias del modelo y el solver:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [16]:
#Generando variables del problema:
talks = pd.read_csv('PyConLatam.csv', ";")

In [17]:
talks

,Charla,Ponentes,Tipo de charla,Turno preferido,Duración,Dias posibles
0,“Phyton: Python para fisica”,Carlangas,Conferencia,tarde,45,"jueves, viernes, sábado"
1,“Introducción a Pandas”,Juanito,Taller,mediodía,60,"jueves, viernes, sábado"
2,“Quantum Machine Learning con Python”,Seth Lloyd,Taller,mañana,60,"jueves, sábado"
3,“Buenas prácticas con NumPy”,Carlangas,Taller,mediodía,30,"jueves, viernes, sábado"
4,“Mejores prácticas con NumBa”,Carlangas,Taller,tarde,30,"jueves, viernes, sábado"
5,“¿Es python una buena idea para el cómputo cie...,"Seth Lloyd, Carlangas",Mesa Redonda,tarde,90,"viernes, sábado"
6,“Testing en cómputo científico y por qué los c...,Pedro Navajas,Conferencia,mediodía,60,"jueves, viernes"
7,“Testing en AI y cómo evitar Skynet”,Sarah Connor,Conferencia,mediodía,60,"sábado, viernes"
8,“Python para la industria 4.0”,El Fede,Conferencia,mañana,45,"jueves, viernes"
9,“Muerte al OOP”,El Barriuc,Conferencia,tarde,60,"viernes, sábado"


In [18]:
events = []
for i in range(len(talks)):
    event = {}
    for j in range(len(talks.loc[i])):
        event.update({talks.columns[j]:talks.loc[i][j]})
    events.append(event)

In [19]:
intervals = {}
active_variables = {}
start = {}
end = {}
#Las charlas son a lo largo de tres días
days = ['jueves', 'viernes', 'sábado']
#Cada día de charlas tiene ocho horas (De 10 AM a 6 PM)
length = 8
total_time = 60*length
#Creamos las variables que representen al proceso
for event in events:
    event_name = event['Charla']
    for day in days:
        start[(event_name,day)] = model.NewIntVar(0,total_time, 'start %s' % event_name)
        duration = event['Duración']
        end[(event_name,day)] = model.NewIntVar(duration,total_time, 'end %s' % event_name)
        active_variables[(event_name,day)] = model.NewBoolVar('active_%s_%s' %(event_name,day))
        intervals[(event_name, day)] = model.NewOptionalIntervalVar(
            start[(event_name,day)], 
            duration, 
            end[(event_name,day)], 
            active_variables[(event_name,day)], 
            '%s_%s' %(event_name, day)
        )

In [20]:
#Cada día, hay un break de una hora para la comida a las 12:00PM y otro de 15 minutos para café a las 4:30PM
breaks = {}
for day in days:
    #Primer break
    breaks[day] = model.NewIntervalVar(2*60,60,3*60, 'Comida %s' %day)
    #Segundo break
    breaks[day] = model.NewIntervalVar(int(6.5*60),15,int(6.75*60), 'Break %s' %day)

In [21]:
'''Cada evento debe ocurrir exactamente una vez'''
for event in events:
    model.Add(sum(active_variables[(event['Charla'], day)] for day in days) == 1)

In [22]:
'''Restringir los días que no pueden ocurrir las pláticas'''
for event in events:
    for day in days:
        if day not in event['Dias posibles']:
            print(day)
            model.Add(active_variables[(event_name,day)] == 0)

viernes
jueves
sábado
jueves
sábado
jueves
viernes


In [10]:
'''Los eventos no deben traslaparse'''
for day in days:
    day_events = [intervals[(event['Charla'],day)] for event in events]
    day_events.append(breaks[day])
    model.AddNoOverlap(day_events)

In [11]:
status = solver.Solve(model)

In [12]:
status

3

In [13]:
if status in [2,4]:
    for day in days:
        print(day)
        for event in events:
            if solver.Value(active_variables[(event['Charla'], day)]):
                event_name = event['Charla']
                print('%s de %i a %i' %(event_name, solver.Value(start[(event_name,day)]), solver.Value(end[(event_name,day)])))
        print()